# Распределенные вычисления ДЗ-2 | Шамаев Онар Евгеньевич 

## Постановка задачи

Необходимо в потоковом формате считывать последнюю активность сообщества r/AskReddit платформы Reddit.
Целью будет найти самые встречающиеся слова.
Необходимо использовать Spark Streaming сохраняя данные на HDFS.

## План
1. Создание TCP сервера (RSS читателя заголовков)
2. Запуск HDFS
3. Написать пайплайн Spark Streaming с окном обработки
4. Результаты

## 1. Создание TCP сервера (RSS читателя заголовков)

Идея технологии RSS заключается в том, чтобы предоставлять последние N текстовых информативных блоков чего-либо. Например на новостных сайтах - это краткая сводка о последних события. В случае Reddit - это список последних назвний последних тем, поднимаемых пользователями в сообществе r/AskReddit.

_Импорты библиотек._

In [1]:
import feedparser
import socket
import time

Последние новости (в RSS формате) доступны по ссылке:
`https://www.reddit.com/r/AskReddit/new/.rss`

In [2]:
rss_link = 'https://www.reddit.com/r/AskReddit/new/.rss'

Определим программу читателя новостной ленты RSS и извлекающей оттуда заголовки, создающий TCP сокет и отправляющий их туда.

In [3]:
encoding = 'utf-8'


def rss2tcp_reader(ip, port, interval_sec: float = 30.0, ttl=None):
    last_stamp: time.struct_time | None = None
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((ip, port))
    s.listen(1)

    print(f'Awaiting connection at {ip}:{port}...')
    conn, addr = s.accept()
    print('Connected by ', addr)

    try:
        while True:
            if (ttl is not None) and last_stamp > ttl:
                print('Time to live passed. Exiting.')
                break

            feed = feedparser.parse(rss_link)
            if feed.status != 200:
                print(f'Bad response {feed.status} received! Exiting.')
                break

            _mx_stmp = None
            for i, entry in enumerate(feed.entries):
                title = entry.title
                time_stamp = entry.published_parsed

                if (last_stamp is None) or (last_stamp < time_stamp):
                    if _mx_stmp is None: _mx_stmp = time_stamp
                else:
                    break

                conn.send(title.encode(encoding) + b'\n')
            if _mx_stmp is not None:
                last_stamp = _mx_stmp

            time.sleep(interval_sec)
    except Exception as ex:
        print(f'Exception happened {ex}')
    finally:
        s.close()
        print(f'Closed connection')

Данная программа представлена отдельно в файле `tcp_server.py`. Запустим ее независмо, чтобы можно было запускать ячейки ниже.

## 2. Запуск HDFS

Воспользуемся наработками 1 дз. Поднимем HDFS кластер из 1 namenode и 2 datanode с помощью Docker.

Запустим кластер из папки `compose` командой `docker-compose -f "docker.compose.yml" up -d`.

UI кластера доступен по адресу `http://localhost:9870/`.

Сама HDFS доступна по адресу `hdfs://localhost:8020`.

In [4]:
hdfs = 'hdfs://localhost:8020'

## 3. Написать пайплайн Spark Streaming с окном обработки

_Импорты библиотек._

In [5]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import os

os.environ['PYSPARK_PYTHON'] = 'python'

Создадим Спарк-контекст и Стриминговый контекст.

In [6]:
from pyspark.sql import SparkSession

ctx = SparkContext(master="local[2]", appName="AskRedditWordCounter")
ctx._jsc.hadoopConfiguration().set('dfs.client.use.datanode.hostname', 'true')

spark = SparkSession(ctx)
spark.conf.set("spark.sql.shuffle.partitions", 5)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57241)
Traceback (most recent call last):
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\socketserver.py", line 755, in __init__
    self.handle()
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\site-packages\pyspark\accumulators.py", line 295, in handle
    poll(accum_updates)
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\site-packages\pyspark\accumulators.py", line 267, in poll
    if self.rfile in r an

In [7]:
ctx_stream = StreamingContext(ctx, 10)

C:\Users\Enzhine\miniconda3\envs\sparked\Lib\site-packages\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


handling store at 2025-03-14 03:45:20
set top to ['what', 'you', 'the', 'is', 'and'] at 2025-03-14 03:45:20
handling store at 2025-03-14 03:45:50
set top to ['you', 'what', 'your', 'the', 'would'] at 2025-03-14 03:45:50
handling store at 2025-03-14 03:46:20
storing
set top to ['you', 'a', 'of', 'that', 'what'] at 2025-03-14 03:46:20
handling store at 2025-03-14 03:46:50
storing
set top to ['what', 'a', 'your', 'the', 'to'] at 2025-03-14 03:46:50
handling store at 2025-03-14 03:47:20
storing
set top to ['you', 'what', 'to', 'the', 'of'] at 2025-03-14 03:47:20
handling store at 2025-03-14 03:47:50
storing
set top to ['you', 'what', 'the', 'do', 'in'] at 2025-03-14 03:47:50
handling store at 2025-03-14 03:48:20
storing
set top to ['you', 'what', 'your', 'the', 'would'] at 2025-03-14 03:48:20
handling store at 2025-03-14 03:48:50
storing
set top to ['what', 'you', 'your', 'the', 'a'] at 2025-03-14 03:48:50
handling store at 2025-03-14 03:49:20
storing
set top to ['you', 'the', 'what', 'how

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Enzhine\miniconda3\envs\sparked\Lib\site-packages\py4j\clientserver.py", line 539, in send_command
    rais

PySpark UI доступен по адресу `http://localhost:4040`.

Будем использовать DStream, считывающий данные по TCP соединению, созданном нашим RSS читателем.

In [8]:
ip = 'localhost'
port = 9999

d_stream = ctx_stream.socketTextStream(ip, port)

Определим фильтр, для подготовки к разбиению заголовков на слова, и непосредственно разделитель на слова.

In [9]:
import string


def prepare(line: str) -> str:
    return line.translate({key: ' ' for key in string.punctuation + '?!()$@/'}).lower()


def word_splitter(line: str) -> list[str]:
    return filter(lambda w: len(w) != 0, line.split(' '))

Определим следующий пайплайн:
1. У потока DStream берем производный поток DStream с окном в 30 секунд. Назовем его потоком заголовков.
2. На поток заголовков поставим обработчики: фильтр всех закголовков, имеющих слова из топ-5 слов, и обработчик сохранения в HDFS заголовков.
3. Сделаем новый производный поток трансформации заголовоков в пары (слово, количество), которые будут сортироваться по количеству в обратном порядке. После сортировки поставим обработчик: сохранения топ-5.

Таким образом каждые 30 секунд, с некоторой фазой, переменная топ-5 заполнится актуальным топом. С интервалом в 30 секунд поток заголовков будет их фильтроват по топу и сохранять в HDFS.

In [10]:
windowed_lines = d_stream.window(windowDuration=30, slideDuration=30)


def store(time_, rdd):
    print(f'handling store at {time_}')
    if rdd.isEmpty():
        return

    print('storing')
    df = rdd.map(lambda line: (line,)).toDF(schema=["titles"])
    df.write.format('json').mode('append').save(hdfs + '/reddit_titles.json')


current_top = []


def filter_by_top(line):
    global current_top

    print(f'checked {line} for contain of {current_top}')
    return any(word in line for word in current_top)

windowed_lines_cached = windowed_lines.transform(lambda rdd: rdd.filter(filter_by_top)).foreachRDD(store)


def handle_top(time_, rdd):
    global current_top
    current_top = rdd.map(lambda pair: pair[0]).take(5)
    print(f'set top to {current_top} at {time_}')


windowed_lines.flatMap(lambda line: word_splitter(prepare(line))) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .transform(lambda rdd: rdd.sortBy(lambda x: -x[1])) \
    .foreachRDD(handle_top)

Запустим джобу на 10 минут. Теперь она будет работать на фоне и собирать статистику встречаемых слов.

In [11]:
ctx_stream.start()
ctx_stream.awaitTermination(10 * 60)

_Импорты библиотек._

In [12]:
import matplotlib.pyplot as plt

# 4. Результаты
Вычитаем полученные заголовки.

In [14]:
df = spark.read.json(hdfs + "/reddit_titles.json")

In [15]:
pandas_df = df.toPandas()
pandas_df.head(10)

,titles
0,Why when a male customer entered the store whe...
1,Why when a male customer entered the store whe...
2,"Women, you marry a man that supports you in yo..."
3,[Serious] US people serving in the armed force...
4,People who were working during the 2008 recess...
5,What is your opinion of a school Parent Organi...
6,"Redditors that believe in ""high value men"" or ..."
7,Have you or do you know anyone who's won a yea...
8,How has the level of stress in your life chang...
9,What would happen if there were an AI that cou...
